In [1]:
import torch
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [25]:
#printing data from json file



# Specify the path to the JSON file
file_path = "data/train.json"

# Open the JSON file
with open(file_path) as file:
    # Load the JSON data
    data = json.load(file)

for element in data:
    print(element)


Politics
Health
Finance
Travel
Food
Education
Environment
Fashion
Science
Sports
Technology
Entertainment


We have three examples per category which is not a lot. The examples are short sentences which makes it almost impossible for td-idf and word2vec to work properly. Training seems difficult so we could try few shot learning for example, we could use a model as encoder and then usethe right distance to compare to the few examples we have. 

In [31]:
#Premiers tests avec TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

# Create a list of sentences from json file 

sentences = []

for element in data:
    for i in data[element]:
        sentences.append(i)

#print(sentences)

['The mayor announced a new initiative to improve public transportation.', 'The senator is facing criticism for her stance on the recent bill.', 'The upcoming election has sparked intense debates among the candidates.', 'Regular exercise and a balanced diet are key to maintaining good health.', 'The World Health Organization has issued new guidelines on COVID-19.', 'A new study reveals the benefits of meditation for mental health.', 'The stock market saw a significant drop following the announcement.', 'Investing in real estate can be a profitable venture if done correctly.', "The company's profits have doubled since the launch of their new product.", 'Visiting the Grand Canyon is a breathtaking experience.', 'The tourism industry has been severely impacted by the pandemic.', 'Backpacking through Europe is a popular choice for young travelers.', 'The new restaurant in town offers a fusion of Italian and Japanese cuisine.', 'Drinking eight glasses of water a day is essential for staying

In [41]:

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)

In [42]:

query = "The role of credit scores in lending decisions is significant."
query_vec = vectorizer.transform([query])
results = cosine_similarity(X,query_vec)



highest_indices = np.argsort(results.flatten())[-5:]
highest_values = results.flatten()[highest_indices]

sorted_indices = np.argsort(results.flatten())
sorted_results = results.flatten()[sorted_indices]

print(sorted_results)
print(highest_values)


for i in highest_indices:
    print(sentences[i])





[0.         0.         0.         0.02763934 0.03092786 0.03299303
 0.03318711 0.04813076 0.06174145 0.06223583 0.06223583 0.07598174
 0.08656039 0.09263897 0.09690515 0.09890436 0.1102567  0.11777686
 0.12890202 0.13902519 0.14353748 0.14738137 0.15277842 0.15362058
 0.15486086 0.17471614 0.18756061 0.20357324 0.20409224 0.21248346
 0.21257964 0.21529027 0.25866692 0.28205742 0.28772251 0.34645899]
[0.21529027 0.25866692 0.28205742 0.28772251 0.34645899]
The annual Met Gala is a major event in the fashion world.
The stock market saw a significant drop following the announcement.
The Grammy Awards are a celebration of the best music of the year.
The latest season of Game of Thrones had fans on the edge of their seats.
Climate change is causing a significant rise in sea levels.


Classification doesn't seem to work properly which seems logical because there are no common words between query and the sentences on which the model has been trained. The "documents" are too short and doesn't contain enough elements to make it work.  Let's try wordTovec on our training set. 

In [ ]:
from gensim.models import Word2Vec
from multiprocessing import cpu_count

cpu = cpu_count()
print('The virtual instance has {} cpus, that will be used to train the word2vec model'.format(cpu))

# We will just get the "WordVectors" parameter from the trained Word2Vec model.
# Otherwise, we could continue training with some more exemples that could be
# fed on the fly to the model.
print("Training the W2V ...")
pol = Word2Vec(, vector_size=100, window=5, min_count=3, workers=cpu)
pol.train(cleaned_pol, total_examples=len(cleaned_pol), epochs=10)
pol_wv = pol.wv


Trying to use a pre-trained model from hugging face as a few shot classifier. 

In [12]:
#Let's make a first try with distilbert 

from transformers import DistilBertForMaskedLM, DistilBertConfig, DistilBertTokenizer, DistilBertModel

config = DistilBertConfig(vocab_size_or_config_json_file = "config.json")

model =  DistilBertModel(config)

dictio = torch.load("pytorch_model.bin", map_location=torch.device('cpu'))

#We remove "distilbert" form the fields names because it is not in the model state_file 
dictio = {k.replace("distilbert.",""):v for k,v in dictio.items()}

#We remove the pre-classifier and classifier weights
dictio = {k:v for k,v in dictio.items() if "pre_classifier" not in k and "classifier" not in k} 



model.load_state_dict(dictio)
model.eval()



DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li